<a href="https://colab.research.google.com/github/jeevannani1705/Project-with-ML/blob/main/Naive_Bayes_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# Code to read csv file into colaboratory:
!pip install sentence-transformers

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import numpy as np
import pandas as pd
import nltk
# nltk.download('punkt')
import sentence_transformers
from sentence_transformers import SentenceTransformer, models
data_set=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROJECT WITH ML 2022-JEEVAN/global-headline-only.csv')[['event_type', 'cleaned']]
data_set.head(1)

,event_type,cleaned
0,Explosions/Remote violence,regime forces shelled Arnaba town in Idleb cou...


In [34]:
print (f"Shape of original data set is {data_set.shape}")

Shape of original data set is (1034527, 2)


In [35]:
df=data_set.sample(200000).reset_index().drop(['index'], axis = 1)
df.head(1)

,event_type,cleaned
0,Protests,workers and employees of the post office suppo...


In [36]:
#del data_set

In [37]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [38]:
train = model.encode(df.cleaned, device='cuda')

In [39]:
from sklearn import preprocessing
# features_trainl encode the target variable
encoder = preprocessing.LabelEncoder()
df["event_type_num"] = encoder.fit_transform(df["event_type"])

In [40]:
df_features = pd.DataFrame()
df_features['cleaned_emb'] = [i for i in train] 
df_features['event_type'] = df["event_type_num"]
df_features.head(3)

,cleaned_emb,event_type
0,"[-0.16274141, 0.25068155, 0.14155233, -0.40031...",2
1,"[-0.067208886, 0.28249285, -0.18293574, 0.2931...",1
2,"[-0.084591076, -0.08829581, 0.22644824, 0.0120...",5


In [41]:
 df_features.to_csv("./features_set.csv")

In [42]:
from sklearn.decomposition import PCA
import time
pca_model = PCA(n_components=50)
start = time.process_time()
pca_model.fit(train)
print("Sum of variance ratios: ",sum(pca_model.explained_variance_ratio_))
print(time.process_time() - start)

Sum of variance ratios:  0.5962399192222281
26.953875272000005


In [43]:
x_comps = pca_model.transform(train)
x_comps.shape

(200000, 50)

**Model Training**

Naive Bayes

In [44]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_comps,df_features['event_type'],test_size=0.2,random_state=42)

In [45]:
from sklearn.naive_bayes import GaussianNB,BernoulliNB

gnb = GaussianNB()
gnb.fit(x_train,y_train)
print("Score of GaussianNB",gnb.score(x_test,y_test))

Score of GaussianNB 0.823425


In [46]:
y_pred = gnb.predict(x_test)

In [47]:
print(y_pred)
print(y_test)

[0 1 0 ... 0 2 0]
119737    0
72272     5
158154    0
65426     5
30074     5
         ..
4174      1
91537     2
156449    0
184376    2
6584      0
Name: event_type, Length: 40000, dtype: int64


In [48]:
from sklearn.metrics import classification_report

print(classification_report(y_true= y_test, y_pred= y_pred))

              precision    recall  f1-score   support

           0       0.80      0.80      0.80      9560
           1       0.85      0.68      0.76      5215
           2       0.96      0.96      0.96     14050
           3       0.65      0.70      0.68      3313
           4       0.65      0.69      0.67      2133
           5       0.70      0.78      0.74      5729

    accuracy                           0.82     40000
   macro avg       0.77      0.77      0.77     40000
weighted avg       0.83      0.82      0.82     40000



K FOLD

In [49]:
# Making the Confusion Matrix 
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred)

In [50]:
# Applying k-Fold Cross Validation 
from sklearn.model_selection import cross_val_score 
accuracies = cross_val_score(estimator = gnb, X = x_train, y = y_train, cv = 10) 
accuracies.mean() 
accuracies.std()

0.003525914110198954

In [51]:
# !pip install matplotlib-venn

In [52]:
# !apt-get -qq install -y libfluidsynth1

In [53]:
# https://pypi.python.org/pypi/libarchive
# !apt-get -qq install -y libarchive-dev && pip install -U libarchive
# import libarchive

In [54]:
# # https://pypi.python.org/pypi/pydot
# !apt-get -qq install -y graphviz && pip install pydot
# import pydot

In [55]:
# !pip install cartopy
# import cartopy

In [56]:
# import pandas as pd
# import sklearn
# from sklearn.feature_selection import SelectFromModel
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.metrics import confusion_matrix
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import train_test_split

In [57]:
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score
# scores = cross_val_score(
# svm_classifier, x_train, y_train, cv=5, scoring='f1_macro')

In [58]:
# from sklearn import model_selection
# from sklearn.model_selection import KFold

In [59]:
# results = model_selection.cross_val_score(svm_classifier,x_train,y_train, cv=KFold)

In [60]:
# # More algorithms!!!!
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import GaussianNB,BernoulliNB

# rfc = RandomForestClassifier()
# rfc.fit(x_train,y_train)
# print("Score of RFC",rfc.score(x_test,y_test))

# logreg = LogisticRegression()
# logreg.fit(x_train,y_train)
# print("Score of LogReg",logreg.score(x_test,y_test))

# gnb = GaussianNB()
# gnb.fit(x_train,y_train)
# print("Score of GaussianNB",gnb.score(x_test,y_test))

# bnb = BernoulliNB()
# bnb.fit(x_train,y_train)
# print("Score of BernoulliNB",bnb.score(x_test,y_test))